# PhaLP

In [2]:
using Random
rand((-1,1), 10000)

10000-element Vector{Int64}:
 -1
 -1
 -1
  1
  1
 -1
 -1
 -1
 -1
  1
  ⋮
  1
  1
 -1
 -1
  1
  1
  1
  1
 -1

In [6]:
using MySQL
conn = DBInterface.connect(MySQL.Connection, "127.0.0.1", "root", "DB_Pw123", db = "PhaLP")

MySQL.Connection(host="127.0.0.1", user="root", port="3306", db="PhaLP")

## A look at the data

In [21]:
using DataFrames
tables = DataFrame(DBInterface.execute(conn, "SHOW TABLES;"))

Row,Tables_in_PhaLP
,String
1,CDSs
2,EC
3,UniParc
4,UniProt
5,domains
6,experimental_evidence
7,gene_ontologies
8,hosts
9,link_EC_UniProt


In [11]:
using DataFrames
hosts = DataFrame(DBInterface.execute(conn, "SELECT * FROM PhaLP.link_phage_host;"))

Row,phages_ID,hosts_ID
,String,String
1,1002724,623
2,1002725,623
3,1003177,670
4,1006972,194
5,1007127,1280
6,1007869,43767
7,1010614,1280
8,1010615,1280
9,101570,90371


## Make dictionaries to map UniProt accessions to their Uniparc ID (UPI) and the other way around


In [10]:
using DataFrames
acc2upi = Dict()
upi2acc = Dict()

data = DataFrame(DBInterface.execute(conn, "SELECT * FROM PhaLP.UniProt;"))

Row,UniProt_ID,name,type,type_evidence,type_probability,UniParc_ID,phages_ID,date_created,date_last_updated
,String,String,String?,String?,Int8?,String,String,Date,Date
1,A0A023J3R2,Lysin,endolysin,ML prediction,95,UPI000009B5E8,1414741,2014-07-09,2020-10-07
2,A0A023MHA8,Endolysin,endolysin,GO annotation,99,UPI000358374C,1446490,2014-07-09,2021-09-29
3,A0A023MHL5,Lysozyme,endolysin,Protein name annotation,98,UPI00025EA4B0,1446489,2014-07-09,2021-09-29
4,A0A023NGE0,Glyco_hydro_19_cat domain-containing protein,endolysin,ML prediction,95,UPI000444C4BC,1472912,2014-07-09,2020-10-07
5,A0A023NGU6,Putative tail structure protein,VAL,ML prediction,55,UPI000444E028,1472912,2014-07-09,2021-09-29
6,A0A023VTC7,Endolysin,endolysin,GO annotation,98,UPI000456C488,1492784,2014-07-09,2021-09-29
7,A0A023W6J3,SLT domain-containing protein,endolysin,ML prediction,61,UPI000456AE9B,1481112,2014-07-09,2020-08-12
8,A0A023W6J8,Lysozyme,endolysin,ML prediction,88,UPI000456A579,1481112,2014-07-09,2021-09-29
9,A0A023W6M9,Lysozyme,VAL,InterPro domain annotation,90,UPI000456BD43,1481112,2014-07-09,2021-09-29


In [23]:
up2type = Dict(zip(data[:, 6], data[:, 3]))
@save "../data/phalp_type.jld" up2type

In [24]:
up_evd_type = Dict(zip(data[:, 6], data[:, 4]))


Dict{String, Union{Missing, String}} with 11549 entries:
  "UPI000178DD30" => "GO annotation"
  "UPI0013EFDC93" => "Protein name annotation"
  "UPI000172D062" => "InterPro domain annotation"
  "UPI001463ABBB" => "ML prediction"
  "UPI000232F56D" => "Protein name annotation"
  "UPI0011625D30" => "InterPro domain annotation"
  "UPI0009882324" => "ML prediction"
  "UPI000CA1D611" => "InterPro domain annotation"
  "UPI0006BC2F8A" => "ML prediction"
  "UPI000BBF7878" => "ML prediction"
  "UPI00138B2696" => "GO annotation"
  "UPI00025D6AED" => "ML prediction"
  "UPI0010C2D3EE" => "ML prediction"
  "UPI000D22144F" => "GO annotation"
  "UPI0018623B24" => "ML prediction"
  "UPI00080F0655" => "ML prediction"
  "UPI00022BD3A3" => "ML prediction"
  "UPI0008093543" => "ML prediction"
  "UPI001463E938" => "ML prediction"
  ⋮               => ⋮

#### Filter out ML based predictions

In [25]:
using JLD
up_evd_type = filter(x ->(last(x) != "ML prediction"), up_evd_type)
@save "../data/phalp_non_ml.jld" up_evd_type

In [26]:
up_evd_type_ML = Dict(zip(data[:, 6], data[:, 4]))
up_evd_type_ML = filter(x-> (last(x) == "ML prediction"), up_evd_type_ML)
@save "../data/phalp_ml.jld" up_evd_type_ML

In [27]:
s = size(data,1)
for row in 1:s
    acc2upi[data[row, 1]]=data[row, 6]
    if data[row, 6] ∉ collect(keys(upi2acc))
        upi2acc[data[row, 6]]=[]
    end
    push!(upi2acc[data[row, 6]], data[row, 1])
end

In [12]:
domacc2domname = Dict()
using CSV

CSV.write("t.csv", DBInterface.execute(conn,"SELECT * FROM `PhaLP`.`link_UniParc_domains` as l join `PhaLP`.`domains` as d where d.`domains_ID` = l.`domains_ID`;"))
data = CSV.read("t.csv", DataFrame)
upi2doms = Dict()

s = size(data,1)
for row in 1:s
    if data[row,1] ∉ collect(keys(domacc2domname))
        domacc2domname[String(data[row,1])]= data[row,8]
    end
    if data[row,2] ∉ collect(keys(upi2doms))
        upi2doms[String(data[row,2])] = []
    end
    push!(upi2doms[String(data[row,2])], data[row,1])
end
domacc2domname

Dict{Any, Any} with 482 entries:
  "SM00490"             => "helicase superfamily c-terminal domain"
  "cd08010"             => "proteins similar to Escherichia coli YceG/mltG may …
  "PF13354"             => "Beta-lactamase enzyme family"
  "PS51724"             => "SPOR domain profile"
  "PTHR43343"           => "PEPTIDASE S12"
  "PF08460"             => "Bacterial SH3 domain"
  "PS51781"             => "SH3b domain profile"
  "PS51186"             => "Gcn5-related N-acetyltransferase (GNAT) domain prof…
  "PTHR47053:SF2"       => "ENDOPEPTIDASE YDDH-RELATED"
  "G3DSA:4.10.80.30"    => "DNA polymerase; domain 6"
  "SSF50685"            => "Barwin-like endoglucanases"
  "PTHR48125:SF4"       => "LP07818P1"
  "G3DSA:3.10.450.190"  => missing
  "PF01520"             => "N-acetylmuramoyl-L-alanine amidase"
  "SSF53955"            => "Lysozyme-like"
  "PF02557"             => "D-alanyl-D-alanine carboxypeptidase"
  "PF03330"             => "Lytic transglycolase"
  "G3DSA:6.10.250.3380" =>

In [13]:
upi2doms

Dict{Any, Any} with 11540 entries:
  "UPI000178DD30" => Any[String31("cd06583"), String31("G3DSA:3.40.80.10"), Str…
  "UPI0013EFDC93" => Any[String31("cd06583"), String31("G3DSA:2.70.70.10"), Str…
  "UPI000172D062" => Any[String31("G3DSA:2.70.70.10"), String31("G3DSA:6.20.110…
  "UPI0011625D30" => Any[String31("cd13402"), String31("G3DSA:1.20.120.20"), St…
  "UPI0009882324" => Any[String31("cd06583"), String31("G3DSA:2.10.270.10"), St…
  "UPI0006BC2F8A" => Any[String31("cd06583"), String31("G3DSA:3.40.80.10"), Str…
  "UPI00025D6AED" => Any[String31("cd00737"), String31("G3DSA:1.10.530.40"), St…
  "UPI000D22144F" => Any[String31("cd13402"), String31("G3DSA:1.10.287.1490"), …
  "UPI0008093543" => Any[String31("G3DSA:3.90.1720.10"), String31("PF05382"), S…
  "UPI0014638858" => Any[String31("G3DSA:3.30.1380.10"), String31("PF13539"), S…
  "UPI0004593892" => Any[String31("G3DSA:1.10.530.10"), String31("PF01832"), St…
  "UPI000178C353" => Any[String31("cd00736"), String31("G3DSA:1.10.530.10"

In [14]:
upi2doms["UPI0013204365"]

4-element Vector{Any}:
 "cd13925"
 "G3DSA:1.10.530.10"
 "PF06737"
 "SSF53955"

In [30]:
using JLD
@save "../data/phalp_domacc2domname.jld" domacc2domname
@save "../data/phalp_upi2doms.jld" upi2doms 

In [31]:
data = DataFrame(DBInterface.execute(conn,"SELECT * FROM PhaLP.UniParc;"))
seq = Dict(zip(data[:, 1],data[:,2]))

Dict{String, Vector{UInt8}} with 11549 entries:
  "UPI000178DD30" => [0x4d, 0x41, 0x4b, 0x56, 0x51, 0x46, 0x54, 0x4b, 0x52, 0x5…
  "UPI0013EFDC93" => [0x4d, 0x47, 0x59, 0x4d, 0x59, 0x50, 0x56, 0x50, 0x4b, 0x4…
  "UPI000172D062" => [0x4d, 0x56, 0x49, 0x4d, 0x53, 0x45, 0x46, 0x56, 0x57, 0x4…
  "UPI001463ABBB" => [0x4d, 0x4e, 0x4c, 0x53, 0x41, 0x4e, 0x46, 0x53, 0x4c, 0x4…
  "UPI000232F56D" => [0x4d, 0x41, 0x4c, 0x47, 0x41, 0x50, 0x4d, 0x45, 0x4e, 0x4…
  "UPI0011625D30" => [0x4d, 0x45, 0x47, 0x47, 0x44, 0x4b, 0x4c, 0x51, 0x47, 0x4…
  "UPI0009882324" => [0x4d, 0x44, 0x49, 0x44, 0x52, 0x4e, 0x52, 0x4c, 0x52, 0x5…
  "UPI000CA1D611" => [0x4d, 0x4b, 0x51, 0x41, 0x41, 0x59, 0x47, 0x53, 0x4c, 0x5…
  "UPI0006BC2F8A" => [0x4d, 0x53, 0x46, 0x4b, 0x4d, 0x4b, 0x59, 0x50, 0x49, 0x4…
  "UPI000BBF7878" => [0x4d, 0x54, 0x45, 0x52, 0x56, 0x4c, 0x52, 0x59, 0x44, 0x4…
  "UPI00138B2696" => [0x4d, 0x51, 0x4c, 0x53, 0x52, 0x4b, 0x47, 0x4c, 0x44, 0x4…
  "UPI00025D6AED" => [0x4d, 0x53, 0x4e, 0x52, 0x4e, 0x49, 0x5

## Embed sequences

### Bag of trimers embeddings with ESM AA

In [32]:
include("../src/HDC.jl")
include("../src/math.jl")
include("../src/experimental.jl")
aa_embeddings = DataFrame(CSV.File("../data/amino_acid_embeddings.csv"))
amino_acids_esm = [only(i) for i in aa_embeddings.protein_ID]
aa_emb = Matrix(aa_embeddings[:, 2:end])
# Create HDVs
HDV_mat_bit = nested_arrays2mat([bithdv() for i in 1:size(aa_emb)[2]], true)

# Extend embeddings into hyperdimensional space
AA_bit_esm = permutedims(mat_scaler(aa_emb * HDV_mat_bit, 0, 1, 2) .|> round)

AA_dict = Dict(zip(amino_acids_esm, [convert(BitVector,AA_bit_esm[:, i]) for i in 1:21]))

Dict{Char, BitVector} with 21 entries:
  'P' => [0, 1, 1, 1, 1, 0, 0, 1, 1, 1  …  0, 0, 0, 1, 0, 0, 1, 1, 0, 1]
  'K' => [1, 1, 0, 1, 0, 1, 1, 1, 1, 0  …  1, 0, 0, 0, 0, 0, 1, 1, 1, 0]
  'M' => [1, 0, 1, 0, 1, 1, 0, 1, 1, 1  …  1, 0, 0, 0, 0, 0, 1, 1, 1, 1]
  'F' => [0, 0, 1, 1, 0, 1, 1, 0, 0, 1  …  1, 1, 0, 1, 1, 0, 0, 1, 0, 1]
  'I' => [0, 0, 1, 1, 0, 0, 1, 0, 1, 1  …  1, 1, 0, 0, 0, 0, 1, 0, 0, 0]
  'H' => [0, 1, 0, 1, 1, 1, 1, 1, 0, 0  …  1, 0, 0, 1, 0, 0, 1, 1, 0, 0]
  'E' => [1, 1, 0, 0, 1, 0, 0, 1, 0, 0  …  0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
  'W' => [0, 1, 0, 1, 1, 0, 0, 0, 0, 1  …  0, 0, 0, 1, 1, 1, 1, 1, 0, 0]
  'S' => [1, 1, 0, 1, 0, 0, 1, 1, 1, 0  …  0, 0, 0, 0, 1, 1, 1, 0, 1, 1]
  'T' => [0, 1, 0, 1, 1, 0, 1, 1, 1, 0  …  0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
  'C' => [0, 0, 0, 1, 0, 0, 1, 1, 0, 0  …  0, 1, 1, 0, 0, 0, 1, 0, 0, 1]
  'X' => [0, 0, 0, 1, 0, 1, 1, 1, 1, 0  …  0, 0, 0, 0, 0, 1, 1, 0, 1, 1]
  'D' => [1, 0, 0, 1, 0, 0, 1, 1, 1, 1  …  0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
  'A' => [0,

In [33]:
trimer_hdvs = Dict(aa1 * aa2 * aa3 => 
bitbind(AA_dict[aa1], circshift(AA_dict[aa2], 1), circshift(AA_dict[aa3], 2)) 
for aa1 in amino_acids_esm for aa2 in amino_acids_esm  for aa3 in amino_acids_esm)

Dict{String, BitVector} with 9261 entries:
  "HTY" => [1, 1, 1, 1, 0, 1, 1, 0, 0, 0  …  1, 0, 0, 0, 0, 0, 1, 1, 1, 0]
  "DRR" => [1, 0, 1, 0, 1, 0, 0, 0, 0, 1  …  0, 1, 1, 0, 1, 1, 0, 1, 1, 1]
  "QAM" => [0, 0, 1, 0, 1, 1, 0, 1, 1, 0  …  0, 0, 1, 0, 1, 1, 1, 0, 0, 0]
  "WNG" => [0, 1, 0, 1, 1, 1, 1, 1, 1, 1  …  0, 1, 1, 1, 0, 0, 0, 0, 0, 1]
  "PPV" => [0, 0, 1, 0, 1, 1, 1, 0, 0, 1  …  0, 0, 1, 0, 1, 0, 0, 1, 1, 1]
  "WMA" => [0, 1, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 1, 1, 1, 1, 0, 0, 0, 1, 1]
  "TKL" => [0, 0, 1, 1, 1, 0, 1, 0, 1, 1  …  0, 0, 1, 1, 0, 0, 1, 0, 1, 0]
  "MSW" => [0, 1, 0, 1, 0, 0, 1, 0, 0, 0  …  1, 0, 0, 0, 0, 0, 1, 1, 0, 1]
  "XTY" => [1, 0, 1, 1, 1, 1, 1, 0, 1, 0  …  0, 0, 0, 1, 0, 1, 1, 0, 0, 1]
  "ETI" => [0, 1, 1, 0, 1, 0, 0, 0, 0, 1  …  0, 0, 1, 0, 0, 0, 0, 1, 1, 0]
  "KLF" => [1, 0, 0, 0, 1, 1, 1, 1, 0, 1  …  1, 1, 0, 1, 0, 1, 0, 0, 1, 1]
  "AGI" => [1, 0, 0, 0, 1, 0, 1, 0, 1, 1  …  1, 0, 1, 0, 0, 1, 0, 1, 1, 1]
  "NMD" => [1, 1, 0, 0, 0, 0, 0, 1, 1, 1  …  1, 0, 0, 1, 

In [1]:
function bow_embedder(sequence)
    l = [trimer_hdvs[sequence[i:i+2]] for i in 1:length(sequence)-2]
    v = bitadd(hcat(l)...)
    return v
end

bow_embedder (generic function with 1 method)

In [35]:
data = DataFrame(DBInterface.execute(conn,"SELECT * FROM PhaLP.UniParc;"))
seq = Dict(zip(data[:, 1],data[:,2]))

embedded_bow_esm = Dict()
k = collect(keys(seq))
Threads.@threads for i in k
    sequence = String(seq[i])
    new_seq = bow_embedder(sequence)
    embedded_bow_esm[i] = new_seq
end

using JLD
@save "../data/phalp_bow_esm.jld" embedded_bow_esm

### Convolutional ESM embeddings

In [36]:
include("../src/HDC.jl")
include("../src/math.jl")
include("../src/experimental.jl")

data = DataFrame(DBInterface.execute(conn,"SELECT * FROM PhaLP.UniParc;"))
seq = Dict(zip(data[:, 1],data[:,2]))
AA_dict = Dict(zip(amino_acids_esm, [convert(BitVector,AA_bit_esm[:, i]) for i in 1:21]))

embedded_CNN_esm = Dict()
k = collect(keys(seq))
Threads.@threads for i in k
    sequence = String(seq[i])
    new_seq = convolved_embedding(sequence, AA_dict)
    embedded_CNN_esm[i] = new_seq
end

@save "../data/phalp_CNN_esm.jld" embedded_CNN_esm

### random AA bow

In [37]:
data = DataFrame(DBInterface.execute(conn,"SELECT * FROM PhaLP.UniParc;"))
seq = Dict(zip(data[:, 1],data[:,2]))

rand_bit = nested_arrays2mat([bithdv() for i in 1:21])
AA_list = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y', 'X']
AA_dict = Dict(zip(AA_list, [convert(BitVector, rand_bit[:, i]) for i in 1:21]))

trimer_hdvs = Dict(aa1 * aa2 * aa3 => 
bitbind(AA_dict[aa1], circshift(AA_dict[aa2], 1), circshift(AA_dict[aa3], 2)) 
for aa1 in AA_list for aa2 in AA_list for aa3 in AA_list)

embedded_rand_bow = Dict()
k = collect(keys(seq))
Threads.@threads for i in k
    sequence = String(seq[i])
    new_seq = bow_embedder(sequence)
    embedded_rand_bow[i] = new_seq
end

@save "../data/phalp_bow_rand.jld" embedded_rand_bow

### Random AA CNN embedding

In [38]:
data = DataFrame(DBInterface.execute(conn,"SELECT * FROM PhaLP.UniParc;"))
seq = Dict(zip(data[:, 1],data[:,2]))

rand_bit = nested_arrays2mat([bithdv() for i in 1:21])
AA_list = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y', 'X']
AA_dict = Dict(zip(AA_list, [convert(BitVector, rand_bit[:, i]) for i in 1:21]))

embedded_rand_CNN = Dict()
k = collect(keys(seq))
Threads.@threads for i in k
    sequence = String(seq[i])
    new_seq = convolved_embedding(sequence, AA_dict)
    embedded_rand_CNN[i] = new_seq
end

@save "../data/phalp_cnn_rand.jld" embedded_rand_CNN